<a href="https://colab.research.google.com/github/KAMRUZZAMAN-RUSSEL/ML/blob/main/Question19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# States and observations
states = ['Low', 'Medium', 'High']
observations = [710, 650, 680]

# Transition probabilities
trans_probs = {
    'Low': {'Low': 0.7, 'Medium': 0.3},
    'Medium': {'Medium': 0.6, 'High': 0.4},
    'High': {'High': 0.8, 'Medium': 0.2}
}

# Emission probabilities for each observation from each state
emission_probs = {
    710: {'Low': 0.1, 'Medium': 0.2, 'High': 0.8},
    650: {'Low': 0.3, 'Medium': 0.8, 'High': 0.1},
    680: {'Low': 0.3, 'Medium': 0.4, 'High': 0.6}
}

# Initial probabilities (uniform)
init_probs = {state: 1/3 for state in states}

# Viterbi algorithm
V = [{}]  # Viterbi matrix
path = {}  # backpointers

# Initialization step
for state in states:
    V[0][state] = init_probs[state] * emission_probs[observations[0]][state]
    path[state] = [state]

# Recursion
for t in range(1, len(observations)):
    V.append({})
    new_path = {}

    for curr_state in states:
        max_prob, prev_st = max(
            ((V[t - 1][prev_state] *
              trans_probs.get(prev_state, {}).get(curr_state, 0) *
              emission_probs[observations[t]][curr_state], prev_state)
             for prev_state in states),
            key=lambda x: x[0]
        )
        V[t][curr_state] = max_prob
        new_path[curr_state] = path[prev_st] + [curr_state]

    path = new_path

# Termination
final_state = max(V[-1], key=lambda state: V[-1][state])
best_path = path[final_state]
best_prob = V[-1][final_state]

# Output
print("Most likely sequence of states:", best_path)
print("Probability of the sequence:", best_prob)


Most likely sequence of states: ['High', 'Medium', 'High']
Probability of the sequence: 0.010240000000000003
